In [ ]:
from pathlib import Path
import nltk as n
import os

cwd = str(Path.home())
masterloc=cwd+"/moviereviews/masterfile.txt"

total_set=[]
test_set=[]
dev_set=[]
train_set=[]

def character_features(characters,review,sn):
    review = review.lower()
    stopwords = n.corpus.stopwords.words('english')
    reviewlist=n.tokenize.word_tokenize(review)
    review = [i for i in review if i not in stopwords]
    tot_occ_char=0
    for i in characters:
        i = i.lower()
        tot_occ_char+=reviewlist.count(i)
    if len(reviewlist)!=0:
        per_tot_occ_char=tot_occ_char/len(reviewlist)
        total_set.append(({"percentage_of_occurence":per_tot_occ_char},sn))
        
        
def char_feature(characters,review):
    review = review.lower()
    stopwords = n.corpus.stopwords.words('english')
    reviewlist=n.tokenize.word_tokenize(review)
    reviewlist = [i for i in reviewlist if i not in stopwords]
    tot_occ_char=0
    for i in characters:
        i = i.lower()
        tot_occ_char+=reviewlist.count(i)
    if len(reviewlist)!=0:
        per_tot_occ_char=tot_occ_char/len(reviewlist)
    return ({"percentage_of_occurence":per_tot_occ_char})

    
def trainmachine(t_set):
    return n.NaiveBayesClassifier.train(train_set)

def testmachine(machine,t_set):
    print("accuracy of the machine is :",n.classify.accuracy(machine,t_set))
    
f1 = open(masterloc)
for i in f1.readlines():
    moviehash=i.rstrip("\n")
    movie_loc=cwd+"/moviereviews/"+moviehash
    
    
    movie_n_loc=movie_loc+"/"+moviehash+".txt"
    movie_n=open(movie_n_loc).readline()
    
    
    movie_s_loc=movie_loc+"/"+moviehash+"_s.txt"
    movie_s=open(movie_s_loc).read()
    
    
    movie_c_loc=movie_loc+"/"+moviehash+"_c.txt"
    movie_c=[j.rstrip("\n") for j in open(movie_c_loc).readlines()]
    
    for i in range(25):
        movie_r_loc=movie_loc+"/"+moviehash+"_"+str(i+1)+".txt"
        if os.path.isfile(movie_r_loc)==True:
            f=open(movie_r_loc)
            reviewnumber,sn=f.readline().rstrip("\n").split(",")
            review=f.readline()
            if review == "":
                continue
            character_features(movie_c,review,sn)
            
tt=len(total_set)
#train_set=total_set[:int(0.4*tt)]
#dev_set=total_set[int(0.4*tt)+1:int(0.7*tt)]
#test_set=total_set[int(0.7*tt)+1:]
    
test_set=total_set[:int(0.3*tt)]
train_set=total_set[int(0.3*tt)+1:int(0.7*tt)]
dev_set=total_set[int(0.7*tt)+1:]
        
    
machine=trainmachine(train_set)
testmachine(machine,test_set)

while True:
    ans=input("do you want to enter a movie? (y/n)").lower()
    if ans=="y":
        charlist=list(input("enter the list of characters in format c1,c2,c3,c4..").split(","))
        review=input("enter the review")
        print(machine.classify(char_feature(charlist,review)))
    elif ans=="n":
        break
    else:
        print("enter a valid option")
f1.close()